## en-tête des requêtes :

In [20]:
import requests

api_url = 'https://development.api.wandeed.com/api/lst/search?offset=0&limit=500'
access_token='Bearer eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJydXRUeHJ5dlltOUVZcGhpRjRxak45ajFvTktLNnU4YUhuM1QySFFSUU5FIn0.eyJleHAiOjE2NjMwNzExMzMsImlhdCI6MTY2MzA1MzEzMywiYXV0aF90aW1lIjoxNjYzMDUzMTMyLCJqdGkiOiJiYmViZDY1ZC00ODgwLTQ3MDUtOTdiOS1kNWNjYjFjOWFjZjIiLCJpc3MiOiJodHRwczovL2RldmVsb3BtZW50LmF1dGgud2FuZGVlZC5jb20vYXV0aC9yZWFsbXMvd2FuZGVlZC1yZWFsbSIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIzMWZmODY3NC1iOGZhLTQyMmYtYWM3NC02YzFjZGI2YTUwZGUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJ3YW5kZWVkLWNsaWVudCIsIm5vbmNlIjoiMDY1MjU1NmMtMjA2Ny00ZTA3LTg2N2ItZTU3YTc4ZGFmNGQxIiwic2Vzc2lvbl9zdGF0ZSI6IjJlNTNhMDJmLWQ0YTQtNDU5OS05YjRhLWI0YjZmMDM2MTY0YyIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly8qLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXBpLndhbmRlZWQuY29tLyoiLCIqIiwiaHR0cHM6Ly8qLmFkbWluLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXV0aC53YW5kZWVkLmNvbSJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy13YW5kZWVkLXJlYWxtIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJzaWQiOiIyZTUzYTAyZi1kNGE0LTQ1OTktOWI0YS1iNGI2ZjAzNjE2NGMiLCJ1dGxfc3BrdXRpbGlzYXRldXIiOjEsInV0bF91dGlsaXNhdGV1cl9yb2xlcyI6IlsxXSIsInV0bF9jcHJlbm9tIjoiR2F5bG9yZCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJ1dGxfc2FwcGFydGVuYW5jZSI6WzIsMzhdLCJ1dGxfY25vbSI6IlBldGl0IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwibG9jYWxlIjoiZnIiLCJnaXZlbl9uYW1lIjoiR2F5bG9yZCIsInV0bF9zZmtpbnN0YW5jZSI6MSwidXRsX3NhZG1pbmlzdHJlciI6W10sIm5hbWUiOiJHYXlsb3JkIFBldGl0IiwidXRsX3Nkcm9pdHNhY2NlcyI6WzYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XSwiZmFtaWx5X25hbWUiOiJQZXRpdCIsImVtYWlsIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwidXNlcl9ncm91cHMiOlsyLDM4LDYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XX0.DGvq43C3ZqBNi7-ZUmrHXm5_iV7nC5gAo-d4cRhdYU8KjsB9N6i4IKxPFPnbalHJpZe9g4MGiZdWGbVYnBstuNcT5ioQOrlZP1BWUmN8Nkq-yYrnL3uHf7VB8mixOAu0b3kpF7r2XoytCvslDFBGaDcWGPJkIdMNFHw3fMKT-smjj1DTP01RuSUc-ItGHn3-CBBDkEocpwTTfBTEDEwcFZ1n69ct1XjYQPuNvKp2PCoje3Y68h6tGSllhOp6ikksx7-inXvMZ6u9ucoshrAkBvb2PLTkQ6cEZxdqz-ymJQsx4N3pxp71-S0xAsSFM8l9JlQ4Ittkmz1ReDkEKi0Vpw'

headers={'Authorization': f'{access_token}',
             'Content-Type':'application/json'}

In [21]:
url = 'http://127.0.0.1:8000/get_data_task_assigner'
res = requests.get(url)

In [22]:
res.json()

{'matrice_projet': {'data': None,
  'status_code': 401,
  'commentaire': 'erreur récupération données sur le serveur.'},
 'dispos_utilisateurs': {'data': None,
  'status_code': 401,
  'commentaire': 'erreur récupération données sur le serveur.'},
 'matrice_competence': {'data': None,
  'status_code': 401,
  'commentaire': 'erreur récupération données sur le serveur.'},
 'taches': {'data': None,
  'status_code': 401,
  'commentaire': 'erreur récupération données sur le serveur.'},
 'STATUT_RECUPERATION_DONNEES': 'ECHEC'}

In [9]:
from modules.task_assigner import *

In [10]:
df_prj, df_cmp, df_tsk, df_dsp = collect_data_from_request(res)

TypeError: 'NoneType' object is not subscriptable

In [5]:
dir(task_assigner
   )

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'add_local_ids_in_dfs',
 'collect_data_from_request',
 'compute_charge_utilisateur',
 'compute_heures_non_assignees',
 'compute_matrice_specialisaton',
 'convert_to_arrays',
 'data_distributor',
 'linprog',
 'make_A_and_b',
 'make_arcs_and_c',
 'make_arcs_and_cost_func',
 'make_list_ids',
 'make_mapping_dicts_extern_to_local',
 'make_mapping_dicts_local_to_extern',
 'make_mat_cmp',
 'make_mat_prj',
 'make_output_dataframe',
 'make_stat_cmp',
 'make_stat_prj',
 'make_stat_tsk',
 'make_stat_utl',
 'make_statistics',
 'make_usefull_mapping_dicts',
 'np',
 'pd',
 'remap_df_out',
 'solution_interpreter',
 'solve_linear_programmation_problem',
 'solve_problem',
 'sub_functions',
 'tools']

In [1018]:
# import librairies
import numpy as np
import pandas as pd

## RECUPERATION DES DONNEES A PARTIR DE LA REQUETE :
df_prj, df_cmp, df_tsk, df_dsp = collect_data_from_request(res)

# FAIT LA LISTE DE TOUS LES IDS (PRJ, CMP, TSK, UTL) CONTENUS DANS LES DONNEES RECUES
id_utl, id_prj, id_cmp, id_tsk = make_list_ids(df_prj, df_cmp, df_tsk, df_dsp)

# CONVERSION DES IDS Wandeed en Identifiant local
utl_to_int, prj_to_int, cmp_to_int, tsk_to_int = make_mapping_dicts_extern_to_local(id_utl, id_prj, id_cmp, id_tsk)

# CONVERSION DES Identifiant local en IDS Wandeed
int_to_utl, int_to_prj, int_to_cmp, int_to_tsk = make_mapping_dicts_local_to_extern(id_utl, id_prj, id_cmp, id_tsk)

# Comptage nombre cmp, utl, tsk, prj.
n_utl = len(id_utl); n_prj = len(id_prj); n_cmp = len(id_cmp); n_tsk = len(id_tsk);


# AJOUT DES VARIABLES LOCALES DANS LES DATAFRAMES
df_prj, df_cmp, df_tsk, df_dsp = add_local_ids_in_dfs(df_prj, df_cmp, df_tsk, df_dsp,
                                                     cmp_to_int, utl_to_int, tsk_to_int, prj_to_int)

# FABRICATION MATRICE PROJET
mat_prj = make_mat_prj(df_prj,n_prj, n_utl)

# FABRICATION MATRICE COMPETENCE
mat_cmp = make_mat_cmp(df_cmp, n_cmp, n_utl)
        
## MOCKING mat_cmp : ######################################################################
mat_cmp = np.random.binomial(n=3,p=0.25, size=mat_cmp.shape)

## MOCKING mat_prj : ######################################################################
mat_prj = np.random.binomial(n=1,p=0.25, size=mat_prj.shape)

        
# FABRICATION DE DICTIONNAIRE UTILES PAR LA SUITE
d_tsk_to_cmp, d_tsk_to_prj, d_tsk_to_lgt, d_utl_to_dsp = make_usefull_mapping_dicts(df_tsk, df_dsp)

# MOCKING pour les dsp utl#################################################################
d_utl_to_dsp = {utl:np.random.randint(15,50)*0.1 for utl in range(n_utl)}
d_utl_to_dsp['not assigned'] = np.sum(list(d_utl_to_dsp.values()))

# FABRICATION DES ARCS RELIANT TACHES A UTILISATEURS POTENTIELS, AINSI QUE FONCTION DE COUT
arcs, cost_func, n_arcs = make_arcs_and_cost_func(n_tsk, n_utl, 
              d_tsk_to_cmp, d_tsk_to_prj,
              mat_cmp, mat_prj)

# FABRICATION DES MATRICES A et B POUR RESOUDRE AX<=B
A, b = make_A_and_b(n_tsk,n_utl,n_arcs,
                 d_tsk_to_lgt, d_utl_to_dsp,
                 arcs)

# RESOLUTION DU PROBLEME DE PROGRAMMATION LINEAIRE
solution_vector, outcome, method = solve_linear_programmation_problem(A, b, cost_func)


# MISE EN FORME DE LA SOLUTION DANS UN DATAFRAME PANDAS
df_out = make_output_dataframe(solution, arcs,
                          d_tsk_to_lgt, d_tsk_to_cmp, d_tsk_to_prj)

# REMAPPING DES ID LOCAUX EN ID WANDEED
df_out = remap_df_out(df_out,
                int_to_tsk, int_to_utl, int_to_prj, int_to_cmp)

## PRODUCTION DE STATISTIQUES D'INTERPRETATION DU RESULTAT

# Production de statistiques par compétences
stat_cmp = make_stat_cmp(df_out)

# Production de statistiques par utilisateur
stat_utl = make_stat_utl(df_out, d_utl_to_dsp, utl_to_int)

# Production de statistiques par tache
stat_tsk = make_stat_tsk(df_out, d_tsk_to_lgt, int_to_tsk)

# Production de statistiques par projet
stat_prj = make_stat_prj(df_out)

# SORTIE API
OUT = {'solution':df_out.to_dict(),
      'statistics_for':{
          'cmp':stat_cmp.to_dict(),
          'utl':stat_utl.to_dict(),
          'tsk':stat_tsk.to_dict(),
          'prj':stat_prj.to_dict()
      }}

C:\Users\antoi\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [1013]:
def test_mapping(df_out):
    assert all(df_out.tsk.apply(lambda x: x in id_tsk)), "Certains id_tsk ne sont pas dans la liste des id_tsk fournis au départ."
    assert all(df_out.prj.apply(lambda x: x in id_prj)), "Certains id_prj ne sont pas dans la liste des id_prj fournis au départ."
    assert all(df_out.cmp.apply(lambda x: x in id_cmp)), "Certains id_cmp ne sont pas dans la liste des id_cmp fournis au départ."
    assert all(df_out.utl.apply(lambda x: x in id_utl +['not assigned'])), "Certains id_utl ne sont pas dans la liste des id_utl fournis au départ."
test_mapping(df_out)

def test_validite_mathematique_solution(df_out, solution_vector):
    assert all(A @ solution_vector <= b + 1e-10), "Math : la solution_vector x ne respecte pas la contrainte Ax <= b."  # + 1e-10 : prise en compte d'une tolérance d'erreur d'arrondi numérique.
    assert all(df_out['duree_assignee'] >=0), "Un nombre d'heure négatif a été assigné."
    assert all(df_out['duree_assignee'] <= df_out['tsk_lgt']), "La durée assignée pour certaines tâches excède celle leur durée."
test_validite_mathematique_solution(df_out, solution_vector)    

def test_stat_cmp(stat_cmp):
    assert all(stat_cmp['total_h_non_assignee'] >=0), "ValueError dans total_h_non_assignee"
    assert all(stat_cmp[~stat_cmp['niveau_cmp_moyen_par_h_realisee'].isna()]['niveau_cmp_moyen_par_h_realisee'] <= 3),"Mauvais calcul du niveau moyen d'exécution dune tache : > 3."
    assert all(stat_cmp[~stat_cmp['niveau_cmp_moyen_par_h_realisee'].isna()]['niveau_cmp_moyen_par_h_realisee'] >= 0),"Mauvais calcul du niveau moyen d'exécution dune tache : < 0."

test_stat_cmp(stat_cmp)

def test_stat_utl(stat_utl):
    assert all(stat_utl.taux_occupation.apply(pd.notna)), "TypeError: les données de taux d'occupation contiennent des NaN."
    assert all(stat_utl.taux_occupation.apply(lambda x: (isinstance(x, float) or isinstance(x, int)) and x >= 0 and x <= 1 ) ), "TypeError: les données de taux d'occupation sont incorrectes."
    assert all(stat_utl.dsp_utl.apply(pd.notna)), "TypeError: les données de disponibilités totales contiennent des NaN."
    assert all(stat_utl.dsp_utl.apply(lambda x: (isinstance(x, float) or isinstance(x, int)) and x >= 0 ) ), "TypeError: les données de disponibilités totales sont incorrectes."
    assert all(stat_utl.dsp_utl.apply(lambda x: x >= 0 )), "TypeError: Certaines données de disponibilités totales sont négatives."
    assert all(stat_utl['total_h_assignees'] <= stat_utl['dsp_utl']  ), "Certains utilisateurs sont trop chargés par rapport à leurs disponibiltés."
    assert all(stat_utl['niveau_moyen_execution_tsk'] <= 3), "Le calcul du niveau moyen d'exécution d'une tache par unité de temps est FAUX."
    assert all(stat_utl['niveau_moyen_execution_tsk'] >= 0), "Le calcul du niveau moyen d'exécution d'une tache par unité de temps est FAUX."
    
test_stat_utl(stat_utl)

def test_stat_prj(stat_prj):
    assert all(stat_prj.temps_total_non_assigne	 >=0), "ValueError dans temps_total_non_assigne"
    assert all(stat_prj.n_missing_cmp_per_prj.apply(lambda x: isinstance(x,int))), "ValueError: Le nombre de compétences manquantes par projet n'est pas toujours un entier."

test_stat_prj(stat_prj)

def test_stat_tsk(stat_tsk):
    assert all(stat_tsk.n_utl_per_tsk.apply(lambda x: isinstance(x,int))), "ValueError: Le nombre d'utilisateurs par tache n'est pas toujours un entier."

test_stat_tsk(stat_tsk)